In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install -q pymorphy2

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 7.1MB/s 


In [0]:
import numpy as np
import pandas as pd
import scipy
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import itertools
import joblib
import time
import os
import re
import json
from tqdm import tqdm_notebook
import gensim
import pymorphy2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [0]:
DIR_PATH = 'drive/My Drive'

In [0]:
dat = pd.read_csv(os.path.join(DIR_PATH, 'avito.csv'))
cat_descr = pd.read_csv(os.path.join(DIR_PATH, 'avito_category.csv'))

In [0]:
dat['category_name'] = dat.category_id.map(dict(zip(cat_descr.category_id, cat_descr.name)))

In [10]:
dat

,item_id,title,description,price,category_id,category_name
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,Для дома и дачи|Мебель и интерьер|Предметы инт...
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,Для дома и дачи|Мебель и интерьер|Столы и стулья
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,Личные вещи|Красота и здоровье|Приборы и аксес...
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,Хобби и отдых|Книги и журналы|Книги
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,Бытовая электроника|Ноутбуки
...,...,...,...,...,...,...
489512,489512,Music MAN JP6 piezo,Гитара новая в жестком кейсе. Цвет чёрный.,115000.0,50,Хобби и отдых|Музыкальные инструменты|Гитары и...
489513,489513,Резолюции и постановления пленумов цк вкпб 1933г,Резолюции и постановления пленумов ЦК ВКПб с ...,4500.0,43,Хобби и отдых|Книги и журналы|Книги
489514,489514,Дверь входная металлическая Китайская,Входная металлическая дверь Кайзер Е40М. Дверь...,2900.0,25,Для дома и дачи|Ремонт и строительство|Двери
489515,489515,Чехол на samsung S6 duos,Продам чехол-книжку на магните.Цена 300 рублей.,300.0,9,Бытовая электроника|Телефоны|Аксессуары|Чехлы ...


In [13]:
data= dat['category_name'].str.split(pat = "|", n = 4,expand = True)
data

,0,1,2,3
0,Для дома и дачи,Мебель и интерьер,"Предметы интерьера, искусство",None
1,Для дома и дачи,Мебель и интерьер,Столы и стулья,None
2,Личные вещи,Красота и здоровье,Приборы и аксессуары,None
3,Хобби и отдых,Книги и журналы,Книги,None
4,Бытовая электроника,Ноутбуки,None,None
...,...,...,...,...
489512,Хобби и отдых,Музыкальные инструменты,Гитары и другие струнные,None
489513,Хобби и отдых,Книги и журналы,Книги,None
489514,Для дома и дачи,Ремонт и строительство,Двери,None
489515,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки


In [0]:
dat['category_1'] = data[0]
dat['category_2'] = data[1]
dat['category_3'] = data[2]
dat['category_4'] = data[3]

In [15]:
dat

,item_id,title,description,price,category_id,category_name,category_1,category_2,category_3,category_4
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,Для дома и дачи|Мебель и интерьер|Предметы инт...,Для дома и дачи,Мебель и интерьер,"Предметы интерьера, искусство",None
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,Для дома и дачи|Мебель и интерьер|Столы и стулья,Для дома и дачи,Мебель и интерьер,Столы и стулья,None
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,Личные вещи|Красота и здоровье|Приборы и аксес...,Личные вещи,Красота и здоровье,Приборы и аксессуары,None
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,Хобби и отдых|Книги и журналы|Книги,Хобби и отдых,Книги и журналы,Книги,None
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,Бытовая электроника|Ноутбуки,Бытовая электроника,Ноутбуки,None,None
...,...,...,...,...,...,...,...,...,...,...
489512,489512,Music MAN JP6 piezo,Гитара новая в жестком кейсе. Цвет чёрный.,115000.0,50,Хобби и отдых|Музыкальные инструменты|Гитары и...,Хобби и отдых,Музыкальные инструменты,Гитары и другие струнные,None
489513,489513,Резолюции и постановления пленумов цк вкпб 1933г,Резолюции и постановления пленумов ЦК ВКПб с ...,4500.0,43,Хобби и отдых|Книги и журналы|Книги,Хобби и отдых,Книги и журналы,Книги,None
489514,489514,Дверь входная металлическая Китайская,Входная металлическая дверь Кайзер Е40М. Дверь...,2900.0,25,Для дома и дачи|Ремонт и строительство|Двери,Для дома и дачи,Ремонт и строительство,Двери,None
489515,489515,Чехол на samsung S6 duos,Продам чехол-книжку на магните.Цена 300 рублей.,300.0,9,Бытовая электроника|Телефоны|Аксессуары|Чехлы ...,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки


In [0]:
stop_words = set(json.load(open(os.path.join(DIR_PATH, 'stopwords-ru.json'), mode='r', encoding='utf-8')))
stop_words_lemm_set = set()

In [0]:
for word in stop_words:
    lemm_word = morph.parse(word)[0].normal_form
    if lemm_word not in stop_words_lemm_set:
        stop_words_lemm_set = stop_words_lemm_set.union([lemm_word])
if "дом" in stop_words_lemm_set:
    stop_words_lemm_set = stop_words_lemm_set.difference(['дом'])
if "книга" in stop_words_lemm_set:
    stop_words_lemm_set = stop_words_lemm_set.difference(['книга'])
if "другой" in stop_words_lemm_set:
    stop_words_lemm_set = stop_words_lemm_set.difference(['другой'])
if "дверь" in stop_words_lemm_set:
    stop_words_lemm_set = stop_words_lemm_set.difference(['дверь'])

In [22]:
stop_words_lemm_set

{'c',
 'а',
 'алло',
 'без',
 'белый',
 'близко',
 'более',
 'большой',
 'будто',
 'бы',
 'бывать',
 'бывь',
 'быть',
 'в',
 'важный',
 'ваш',
 'вверх',
 'вдали',
 'вдруг',
 'ведь',
 'везде',
 'вернуться',
 'весь',
 'вечер',
 'взгляд',
 'взять',
 'вид',
 'видеть',
 'вместе',
 'вне',
 'вниз',
 'внизу',
 'вод',
 'война',
 'вокруг',
 'вон',
 'вообще',
 'вопрос',
 'восемнадцать',
 'восемь',
 'вот',
 'впрочем',
 'время',
 'все ещё',
 'всегда',
 'всего',
 'всеять',
 'всюду',
 'вы',
 'выйти',
 'где',
 'главный',
 'глаз',
 'говорить',
 'год',
 'голова',
 'голос',
 'город',
 'да',
 'давать',
 'давно',
 'даже',
 'далеко',
 'далёкий',
 'даром',
 'дать',
 'два',
 'двадцать',
 'двенадцать',
 'девятнадцать',
 'девять',
 'действительно',
 'делать',
 'дело',
 'день',
 'деньга',
 'десять',
 'для',
 'до',
 'довольно',
 'долго',
 'должный',
 'дорога',
 'друг',
 'друго',
 'думать',
 'душить',
 'е',
 'если',
 'есть',
 'ещё',
 'её',
 'ж',
 'ждать',
 'же',
 'жена',
 'женщина',
 'жизнь',
 'жить',
 'за',
 'зан

In [24]:
all_categories = dat.category_1.value_counts().keys().tolist() + \
dat.category_2.value_counts().keys().tolist() + \
dat.category_3.value_counts().keys().tolist() + \
dat.category_4.value_counts().keys().tolist()
cat_prep_cat_dict = {}
regexp_pattern = r'[_,.!><?"@#$%^&*\t():»«+-0123456789]'
for category in all_categories:
    sentence = re.sub(regexp_pattern, ' ', category.lower()).split()
    prep_category = ''
    for word in sentence:
        lemm_word = morph.parse(word)[0].normal_form
        if lemm_word not in stop_words_lemm_set:
            prep_category += ' ' + lemm_word
    cat_prep_cat_dict[category] = prep_category.strip()
    
dat['category_1_prep'] = dat['category_1'].map(cat_prep_cat_dict)
dat['category_2_prep'] = dat['category_2'].map(cat_prep_cat_dict)
dat['category_3_prep'] = dat['category_3'].map(cat_prep_cat_dict)
dat['category_4_prep'] = dat['category_4'].map(cat_prep_cat_dict)

lemm_name = []
for full_category_name in tqdm_notebook(dat.category_name):
    current_lemm_name = ''
    for category in full_category_name.split('|'):
        current_lemm_name += cat_prep_cat_dict[category] + ' '
    lemm_name.append(current_lemm_name.strip())
dat['category_name_prep'] = lemm_name

In [25]:
dat

,item_id,title,description,price,category_id,category_name,category_1,category_2,category_3,category_4,category_1_prep,category_2_prep,category_3_prep,category_4_prep,category_name_prep
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,Для дома и дачи|Мебель и интерьер|Предметы инт...,Для дома и дачи,Мебель и интерьер,"Предметы интерьера, искусство",None,дом дача,мебель интерьер,предмет интерьер искусство,NaN,дом дача мебель интерьер предмет интерьер иску...
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,Для дома и дачи|Мебель и интерьер|Столы и стулья,Для дома и дачи,Мебель и интерьер,Столы и стулья,None,дом дача,мебель интерьер,стул,NaN,дом дача мебель интерьер стул
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,Личные вещи|Красота и здоровье|Приборы и аксес...,Личные вещи,Красота и здоровье,Приборы и аксессуары,None,личный вещий,красота здоровье,прибор аксессуар,NaN,личный вещий красота здоровье прибор аксессуар
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,Хобби и отдых|Книги и журналы|Книги,Хобби и отдых,Книги и журналы,Книги,None,хобби отдых,книга журнал,книга,NaN,хобби отдых книга журнал книга
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,Бытовая электроника|Ноутбуки,Бытовая электроника,Ноутбуки,None,None,бытовой электроника,ноутбук,NaN,NaN,бытовой электроника ноутбук
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489512,489512,Music MAN JP6 piezo,Гитара новая в жестком кейсе. Цвет чёрный.,115000.0,50,Хобби и отдых|Музыкальные инструменты|Гитары и...,Хобби и отдых,Музыкальные инструменты,Гитары и другие струнные,None,хобби отдых,музыкальный инструмент,гитара другой струнный,NaN,хобби отдых музыкальный инструмент гитара друг...
489513,489513,Резолюции и постановления пленумов цк вкпб 1933г,Резолюции и постановления пленумов ЦК ВКПб с ...,4500.0,43,Хобби и отдых|Книги и журналы|Книги,Хобби и отдых,Книги и журналы,Книги,None,хобби отдых,книга журнал,книга,NaN,хобби отдых книга журнал книга
489514,489514,Дверь входная металлическая Китайская,Входная металлическая дверь Кайзер Е40М. Дверь...,2900.0,25,Для дома и дачи|Ремонт и строительство|Двери,Для дома и дачи,Ремонт и строительство,Двери,None,дом дача,ремонт строительство,дверь,NaN,дом дача ремонт строительство дверь
489515,489515,Чехол на samsung S6 duos,Продам чехол-книжку на магните.Цена 300 рублей.,300.0,9,Бытовая электроника|Телефоны|Аксессуары|Чехлы ...,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,бытовой электроника,телефон,аксессуар,чехол плёнка,бытовой электроника телефон аксессуар чехол пл...


In [26]:
dat['title_prep'] = None
dat['description_prep'] = None
title_prep_list = []
regexp_pattern = r'[_,.!><?"@#$%^&*\t():»«+-0123456789]'
for i in tqdm_notebook(range(len(dat))):
    sentence = re.sub(regexp_pattern, ' ', dat.title[i].lower()).split()
    current_sentense = ''
    for word in sentence:
        lemm_word = morph.parse(word)[0].normal_form
        if lemm_word not in stop_words_lemm_set:
            current_sentense += ' ' + lemm_word
    title_prep_list.append(current_sentense.strip())
dat.title_prep = title_prep_list

description_prep_list = []
for i in tqdm_notebook(range(len(dat))):
    sentence = re.sub(regexp_pattern, ' ', dat.description[i].lower()).split()
    current_sentense = ''
    for word in sentence:
        lemm_word = morph.parse(word)[0].normal_form
        if lemm_word not in stop_words_lemm_set:
            current_sentense += ' ' + lemm_word
    description_prep_list.append(current_sentense.strip())
dat.description_prep = description_prep_list
prep_columns = np.array([col for col in dat.columns if 'prep' in col or col == 'category_id'])

In [0]:
# Ваш код здесь:
np.random.seed(10) # Не меняем random seed!
tr_indexes, val_indexes, te_indexes = np.split(
    np.random.permutation(np.arange(489517)), [int(489517*0.6), int(489517*0.9)]
)

In [75]:
texts_without_none = np.array([len(sentence) != 0 for sentence in dat['description_prep']])
print(str(sum(~texts_without_none)))

248


In [0]:
dat = dat.loc[texts_without_none]
dat.index = range(dat.shape[0])
dat.title_prep = dat.title_prep.astype(str)
dat.description_prep = dat.description_prep.astype(str)

In [47]:
dat

,item_id,title,description,price,category_id,category_name,category_1,category_2,category_3,category_4,category_1_prep,category_2_prep,category_3_prep,category_4_prep,category_name_prep,title_prep,description_prep
0,0,Картина,Гобелен. Размеры 139х84см.,1000.0,19,Для дома и дачи|Мебель и интерьер|Предметы инт...,Для дома и дачи,Мебель и интерьер,"Предметы интерьера, искусство",None,дом дача,мебель интерьер,предмет интерьер искусство,NaN,дом дача мебель интерьер предмет интерьер иску...,картина,гобелен размер х сантиметр
1,1,Стулья из прессованной кожи,Продам недорого 4 стула из светлой прессованно...,1250.0,22,Для дома и дачи|Мебель и интерьер|Столы и стулья,Для дома и дачи,Мебель и интерьер,Столы и стулья,None,дом дача,мебель интерьер,стул,NaN,дом дача мебель интерьер стул,стул прессовать кожа,продать недорого стул светлый прессовать кожа ...
2,2,Домашняя мини баня,"Мини баня МБ-1(мини сауна), предназначена для ...",13000.0,37,Личные вещи|Красота и здоровье|Приборы и аксес...,Личные вещи,Красота и здоровье,Приборы и аксессуары,None,личный вещий,красота здоровье,прибор аксессуар,NaN,личный вещий красота здоровье прибор аксессуар,домашний минь баня,минь баня мб минь сауна предназначить принятие...
3,3,"Эксклюзивная коллекция книг ""Трансаэро"" + подарок","Продам эксклюзивную коллекцию книг, выпущенную...",4000.0,43,Хобби и отдых|Книги и журналы|Книги,Хобби и отдых,Книги и журналы,Книги,None,хобби отдых,книга журнал,книга,NaN,хобби отдых книга журнал книга,эксклюзивный коллекция книга трансаэро подарок,продать эксклюзивный коллекция книга выпустить...
4,4,Ноутбук aser,Продаётся ноутбук ACER e5-511C2TA. Куплен в ко...,19000.0,1,Бытовая электроника|Ноутбуки,Бытовая электроника,Ноутбуки,None,None,бытовой электроника,ноутбук,NaN,NaN,бытовой электроника ноутбук,ноутбук aser,продаваться ноутбук acer e ta купить ноябрь по...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489441,489512,Music MAN JP6 piezo,Гитара новая в жестком кейсе. Цвет чёрный.,115000.0,50,Хобби и отдых|Музыкальные инструменты|Гитары и...,Хобби и отдых,Музыкальные инструменты,Гитары и другие струнные,None,хобби отдых,музыкальный инструмент,гитара другой струнный,NaN,хобби отдых музыкальный инструмент гитара друг...,music man jp piezo,гитара жёсткий кейс цвета чёрный
489442,489513,Резолюции и постановления пленумов цк вкпб 1933г,Резолюции и постановления пленумов ЦК ВКПб с ...,4500.0,43,Хобби и отдых|Книги и журналы|Книги,Хобби и отдых,Книги и журналы,Книги,None,хобби отдых,книга журнал,книга,NaN,хобби отдых книга журнал книга,резолюция постановление пленум цк вкпб,резолюция постановление пленум цк вкпб сборник...
489443,489514,Дверь входная металлическая Китайская,Входная металлическая дверь Кайзер Е40М. Дверь...,2900.0,25,Для дома и дачи|Ремонт и строительство|Двери,Для дома и дачи,Ремонт и строительство,Двери,None,дом дача,ремонт строительство,дверь,NaN,дом дача ремонт строительство дверь,дверь входной металлический китайский,входной металлический дверь кайзер дверь подхо...
489444,489515,Чехол на samsung S6 duos,Продам чехол-книжку на магните.Цена 300 рублей.,300.0,9,Бытовая электроника|Телефоны|Аксессуары|Чехлы ...,Бытовая электроника,Телефоны,Аксессуары,Чехлы и плёнки,бытовой электроника,телефон,аксессуар,чехол плёнка,бытовой электроника телефон аксессуар чехол пл...,чехол samsung s duos,продать чехол книжка магнит цена рубль


In [49]:
w2v_fpath = os.path.join(DIR_PATH, 'all.norm-sz100-w10-cb0-it1-min100.w2v')
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
word_idf_dict = dict(sorted(dict(zip(tf_idf.get_feature_names(), tf_idf.idf_)).items(),
                            key=lambda x: x[1], reverse=False))

In [53]:
sifted_description = []
for desc in tqdm_notebook(dat.description_prep):
    sifted_description.append(' '.join([word for word in desc.split() if word in tf_idf.vocabulary_]))
dat['description_prep'] = sifted_description

In [54]:
titles_w2v = np.zeros((dat.shape[0], 100))
descr_w2v = np.zeros((dat.shape[0], 100))
cat_w2v = np.zeros((dat.category_name_prep.value_counts().shape[0], 100))
category_names_order = dat.category_name_prep.value_counts().keys().values
for i in tqdm_notebook(range(dat.shape[0])):
    title = str(dat.title_prep[i]).split()
    descr = str(dat.description_prep[i]).split()
    
    denominator = np.array([word_idf_dict.get(word, 0) for word in title])
    numerator = np.array([w2v[word]*word_idf_dict.get(word, 0) if word in w2v else np.zeros((100,)).astype(float) for word in title])
    titles_w2v[i] = numerator.sum(axis=0)/(denominator.sum() + 1e-120)
    
    denominator = np.array([word_idf_dict.get(word, 0) for word in descr])
    numerator = np.array([w2v[word]*word_idf_dict.get(word, 0) if word in w2v else np.zeros((100,)).astype(float) for word in descr])
    descr_w2v[i] = numerator.sum(axis=0)/(denominator.sum() + 1e-120)
    
for n, categ in enumerate(category_names_order):
    denominator = np.array([word_idf_dict.get(word, 0) for word in str(categ).split()])
    numerator = np.array([w2v[word]*word_idf_dict.get(word, 0) if word in w2v else np.zeros((100,)).astype(float) for word in str(categ).split()])
    cat_w2v[n] = numerator.sum(axis=0)/(denominator.sum() + 1e-120)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [55]:
title_cat_dist = scipy.spatial.distance.cdist(titles_w2v, cat_w2v, metric='cosine')
title_descr_dist = []
for i in tqdm_notebook(range(len(dat))):
    dis = scipy.spatial.distance.cosine(titles_w2v[i].astype(float), descr_w2v[i].astype(float))
    if not np.isnan(dis):
        title_descr_dist.append(dis)
    else:
        title_descr_dist.append(-9999.0)
descr_cat_dist = scipy.spatial.distance.cdist(descr_w2v, cat_w2v, metric='cosine')

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [0]:
# разделяем на признаки/таргеты
X = np.hstack((title_cat_dist, np.array(title_descr_dist).reshape(-1, 1), descr_cat_dist))
y = dat.category_id.values
tr_indexes = tr_indexes[tr_indexes<488323-248]
te_indexes = te_indexes[te_indexes<488323-248]
val_indexes = val_indexes[val_indexes<488323-248]
tr_nonnull = tr_indexes[~np.isnan(X[tr_indexes].sum(axis=1))]
te_nonnull = te_indexes[~np.isnan(X[te_indexes].sum(axis=1))]
val_nonnull = val_indexes[~np.isnan(X[val_indexes].sum(axis=1))]
X_tr = X[tr_nonnull]
y_tr = y[tr_nonnull]
X_te = X[np.union1d(te_nonnull, val_nonnull)]
y_te = y[np.union1d(te_nonnull, val_nonnull)]
X_val = X[val_nonnull]
y_val = y[val_nonnull]

In [0]:
dat.title_prep = dat.title_prep.astype(str)
dat.description_prep = dat.description_prep.astype(str)

tf_idf_descr = TfidfVectorizer(min_df=20, max_df=0.9)
tf_idf_descr.fit(dat.description_prep[tr_nonnull])
tf_idf_descr_tr = tf_idf_descr.transform(dat.description_prep[tr_nonnull])

tf_idf_title = TfidfVectorizer(min_df=15, max_df=0.95)
tf_idf_title.fit(dat.title_prep[tr_nonnull])
tf_idf_title_tr = tf_idf_title.transform(dat.title_prep[tr_nonnull])

tf_idf_title_val = tf_idf_title.transform(dat.title_prep[val_nonnull])
tf_idf_descr_val = tf_idf_descr.transform(dat.description_prep[val_nonnull])

tf_idf_title_test = tf_idf_title.transform(dat.title_prep[te_nonnull])
tf_idf_descr_test = tf_idf_descr.transform(dat.description_prep[te_nonnull])

In [0]:
class AVITO(nn.Module):
    def __init__(self, title_dim, description_dim, distances_dim, dense_title_out, dense_description_out, dense_dist_out, n_classes):
        super(AVITO, self).__init__()
        self.title_dim = title_dim
        self.description_dim = description_dim
        self.linear_title = nn.Linear(title_dim, dense_title_out)
        self.linear_description = nn.Linear(description_dim, dense_description_out)
        self.linear_last = nn.Linear(dense_title_out+dense_description_out, n_classes)
        
    def forward(self, title_tfidf, descr_tfidf):
        x1 = self.linear_title(title_tfidf)
        x2 = self.linear_description(descr_tfidf)
        x_concat = torch.cat([x1, x2], dim=1)
        out = self.linear_last(x_concat)
        return out

In [0]:
TITLE_DIM = tf_idf_title_tr.shape[1]
DESCRIPTION_DIM = tf_idf_descr_tr.shape[1]
DISTANCES_DIM = 0
N_CLASSES = dat.category_id[tr_indexes].value_counts().shape[0]

In [0]:
#задаем гиперпараметры модели
model_nn = AVITO(title_dim=TITLE_DIM, description_dim=DESCRIPTION_DIM, distances_dim=DISTANCES_DIM, 
                     dense_title_out=3000, dense_description_out=5000, dense_dist_out=50, n_classes=N_CLASSES)
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model_nn.parameters(), lr=1e-4)
reduce_plato = optim.lr_scheduler.ReduceLROnPlateau(opt, patience=4, factor=0.01)

In [71]:
EPOCHS = 7
BATCH_SIZE = 120
DEVICE = 'cuda'
model_nn.to(DEVICE)
criterion.to(DEVICE)
train_loss = []
test_loss = []
f1_train = []
f1_test = []
for i in range(1, EPOCHS+1):
    model_nn.to(DEVICE)
    model_nn.train()
    local_train_loss = 0
    st = time.perf_counter()
    random_indexes_permutation = np.split(np.random.permutation(tr_nonnull.shape[0]), 
                                           [k*BATCH_SIZE for k in range(1, tr_nonnull.shape[0]//BATCH_SIZE)])
    for batch_indexes in tqdm_notebook(random_indexes_permutation):
        opt.zero_grad()
        x1 = scipy.sparse.coo_matrix(tf_idf_title_tr[batch_indexes])
        inds = torch.LongTensor(np.vstack((x1.row, x1.col)).astype(int))
        vals = torch.FloatTensor(x1.data)
        shape_ = torch.Size(x1.shape)
        x1 = torch.sparse.FloatTensor(inds, vals, shape_).to(DEVICE)

        x2 = scipy.sparse.coo_matrix(tf_idf_descr_tr[batch_indexes])
        inds = torch.LongTensor(np.vstack((x2.row, x2.col)).astype(int))
        vals = torch.FloatTensor(x2.data)
        shape_ = torch.Size(x2.shape)
        x2 = torch.sparse.FloatTensor(inds, vals, shape_).to(DEVICE)

        y_pred = model_nn(x1, x2)
        loss = criterion(y_pred, torch.LongTensor(y_tr[batch_indexes]).to(DEVICE))
        local_train_loss += loss.item()/len(random_indexes_permutation)
        loss.backward()
        opt.step()

    train_loss.append(local_train_loss)
    
    with torch.no_grad():
        model_nn.eval()
        local_test_loss = 0
        random_indexes_permutation = np.split(np.random.permutation(val_nonnull.shape[0]), 
                                           [k*BATCH_SIZE for k in range(1, val_nonnull.shape[0]//BATCH_SIZE)])                        
        for batch_indexes in tqdm_notebook(random_indexes_permutation):
            opt.zero_grad()
            x1 = scipy.sparse.coo_matrix(tf_idf_title_val[batch_indexes])
            inds = torch.LongTensor(np.vstack((x1.row, x1.col)).astype(int))
            vals = torch.FloatTensor(x1.data)
            shape_ = torch.Size(x1.shape)
            x1 = torch.sparse.FloatTensor(inds, vals, shape_).to(DEVICE)

            x2 = scipy.sparse.coo_matrix(tf_idf_descr_val[batch_indexes])
            inds = torch.LongTensor(np.vstack((x2.row, x2.col)).astype(int))
            vals = torch.FloatTensor(x2.data)
            shape_ = torch.Size(x2.shape)
            x2 = torch.sparse.FloatTensor(inds, vals, shape_).to(DEVICE)

            y_pred = model_nn(x1, x2)
            loss = criterion(y_pred, torch.LongTensor(y_val[batch_indexes]).to(DEVICE))
            local_test_loss += loss.item()/len(random_indexes_permutation)

        test_loss.append(local_test_loss)
        reduce_plato.step(local_test_loss)
        
        
    torch.save(model_nn.cpu().state_dict(), os.path.join(DIR_PATH, 'epoch{}'.format(i)))
                    
        
    print('Epoch:', i, 
          'Train loss: {:.4f}'.format(train_loss[-1]), 'Test loss: {:.4f}'.format(test_loss[-1]), 
          'Time: {:.2f}'.format(time.perf_counter() - st))

Epoch: 1 Train loss: 0.9472 Test loss: 0.4385 Time: 81.60


Epoch: 2 Train loss: 0.3574 Test loss: 0.4022 Time: 80.74


Epoch: 3 Train loss: 0.2845 Test loss: 0.4020 Time: 80.30


Epoch: 4 Train loss: 0.2373 Test loss: 0.4140 Time: 80.89


Epoch: 5 Train loss: 0.2029 Test loss: 0.4335 Time: 81.00


Epoch: 6 Train loss: 0.1774 Test loss: 0.4566 Time: 80.53


Epoch: 7 Train loss: 0.1580 Test loss: 0.4822 Time: 80.77


In [72]:
#хорошо ли работает обученная нейросетка
DEVICE = 'cpu'
y_true = []
y_pred = []
for ind in tqdm_notebook(te_nonnull):
  if ind in dat.index:
    descr, tit, cat = dat.loc[ind, ['description_prep', 'title_prep', 'category_id']].values
    y_true.append(cat)
    vec_descr = tf_idf_descr.transform([descr])
    vec_title = tf_idf_title.transform([tit])
    x1 = scipy.sparse.coo_matrix(vec_title)
    inds = torch.LongTensor(np.vstack((x1.row, x1.col)).astype(int))
    vals = torch.FloatTensor(x1.data)
    shape_ = torch.Size(x1.shape)
    x1 = torch.sparse.FloatTensor(inds, vals, shape_).to(DEVICE)

    x2 = scipy.sparse.coo_matrix(vec_descr)
    inds = torch.LongTensor(np.vstack((x2.row, x2.col)).astype(int))
    vals = torch.FloatTensor(x2.data)
    shape_ = torch.Size(x2.shape)
    x2 = torch.sparse.FloatTensor(inds, vals, shape_).to(DEVICE)

    y_pred_current = model_nn(x1, x2).detach().numpy()[0]
    y_pred.append(np.argmax(y_pred_current))

In [73]:
accuracy_score(y_true=y_true, y_pred=y_pred)

0.8658025243173063